In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import pickle

In [3]:
df = pd.read_csv("output.csv")

In [4]:
x= np.array(df["Station Name"])
y= np.array(df["Day"])
z= np.array(df["Time"])
w = np.array(df["Weather"])

In [5]:
print(np.unique(x))

['AIIMS' 'Adarsh Nagar' 'Akshardham' 'Anand Vihar' 'Arjan Garh'
 'Ashok Park Main' 'Azadpur' 'Badarpur Border' 'Botanical Garden'
 'Central Secretariat' 'Chandni Chowk' 'Chawri Bazar' 'Chhatarpur'
 'Civil Lines' 'Dwarka' 'GTB Nagar' 'Ghitorni' 'Green Park'
 'Guru Dronacharya' 'HUDA City Centre' 'Haiderpur' 'Hauz Khas'
 'IFFCO Chowk' 'IITD' 'INA' 'Inderlok' 'Jahangirpuri' 'Janakpuri West'
 'Jor Bagh' 'Kalkaji Mandir' 'Karkarduma' 'Kashmere Gate' 'Kaushambi'
 'Kirti Nagar' 'Lajpat Nagar' 'Laxmi Nagar' 'Lok Kalyan Marg' 'MG Road'
 'Mandi House' 'Mayur Vihar -I' 'Mayur Vihar Extension' 'Model Town'
 'Munirka' 'Netaji Subhash Place' 'New Ashok Nagar' 'New Delhi'
 'Nirman Vihar' 'Noida City Centre' 'Noida Golf Course' 'Noida Sector 15'
 'Noida Sector 16' 'Noida Sector 18' 'Patel Chowk' 'Preet Vihar'
 'Qutab Minar' 'Rajiv Chowk' 'Rithala' 'Rohini Sector 18-19' 'Saket'
 'Samaypur Badli' 'Shahdara' 'Sikandarpur' 'Sultanpur' 'Surajmal Stadium'
 'Terminal 1 IGI Airport' 'Udyog Bhawan' 'Vaishali' 

In [6]:
print(np.unique(y))

['FRIDAY' 'MONDAY' 'SATURDAY' 'SUNDAY' 'THURSDAY' 'TUESDAY' 'WEDNESDAY']


In [7]:
print(np.unique(z))

['10:00' '11:00' '12:00' '1:00' '2:00' '3:00' '4:00' '5:00' '6:00' '7:00'
 '8:00']


In [8]:
print(np.unique(w))

['Cool' 'Rainy' 'Sunny' 'ThunderStorm']


In [9]:
def try_int(x):
    try:
        return int(x)
    except ValueError:
        return 

df["Weather"] = pd.Categorical(df["Weather"]).codes
df["Day"] = pd.Categorical(df["Day"]).codes
df["Station Name"] = pd.Categorical(df["Station Name"]).codes
df["Time"] = df["Time"].map(lambda x: try_int(x.split(":")[0]))
df["Population"] = df["Population"].map(lambda x: try_int(x))

In [10]:
x = df[["Station Name", "Day", "Time", "Weather"]].values
y = df[["Population"]].values

In [11]:
# x = np.nan_to_num(x)
# y = np.nan_to_num(y)
# np.any(np.isnan(y))

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.35, random_state=100)

In [13]:
RandomForesRegModel = RandomForestRegressor(n_estimators = 6, random_state = 0)
RandomForesRegModel.fit(x_train.reshape(-1,4), y_train.ravel())

RandomForestRegressor(n_estimators=6, random_state=0)

In [14]:
y_pred = RandomForesRegModel.predict(x_test.reshape(-1,4))

In [15]:
y_pred[1:10].reshape(-1,1)

array([[ 503.20840032],
       [  55.2864979 ],
       [ 560.1240209 ],
       [  57.03893661],
       [1294.32950118],
       [ 679.3053602 ],
       [  78.2089821 ],
       [1516.62652814],
       [ 685.61685887]])

In [16]:
y_test[1:10].reshape(-1,1)

array([[ 526],
       [  65],
       [ 562],
       [  34],
       [1366],
       [ 749],
       [ 100],
       [1523],
       [ 726]])

In [17]:
print('Accuracy : ', r2_score(y_test, y_pred.reshape(-1,1))*100)
print('MSE: ', mean_squared_error(y_test, y_pred.reshape(-1,1)))

Accuracy :  98.34992004802685
MSE:  3091.5280693169216


In [18]:
PKL_filename = "RFR_Model.pkl"

with open(PKL_filename, 'wb') as file:
    pickle.dump(RandomForesRegModel, file)

In [19]:
with open ("RFR_Model.pkl", 'rb') as file:
    Loaded_Model = pickle.load(file)
Loaded_Model
test_input=np.array([1,6,10,2]) # "Station Name[0-137]", "Day[0-7]", "Time[10-8]", "Weather[0-4]"
result = Loaded_Model.predict(test_input.reshape(-1,4))
result[0]

1174.7244008714597

In [20]:
# score = Loaded_Model.score(x_test.reshape(-1,4),y_test)
# score